In [1]:
# Import required modules
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

week_num = 8

#https://bettingdata.com/nfl/odds?week=5&season=2021&seasontype=1&state=WORLD&league=nfl
df_team_index = pd.read_csv("Team Index.csv")
bets_df = pd.read_csv("BettingOdds.csv")

bets_df['FAVORITE_SCORE'] = bets_df['TOTAL']/2 - bets_df['SPREAD']
bets_df['UNDERDOG_SCORE'] = bets_df['TOTAL']/2 + bets_df['SPREAD']
bets_df['FAVORITE'] = bets_df['FAVORITE'].str.replace("at ", "")
bets_df['UNDERDOG'] = bets_df['UNDERDOG'].str.replace("at ", "")

test_df = bets_df[bets_df['WEEK']==week_num+1]
test_df1 = test_df[['FAVORITE','UNDERDOG','SPREAD','FAVORITE_SCORE']].rename(columns={"FAVORITE": "OFF_TEAM", "UNDERDOG": "DEF_TEAM", "FAVORITE_SCORE": "BETS_SCORE"})
test_df2 = test_df[['FAVORITE','UNDERDOG','SPREAD','UNDERDOG_SCORE']].rename(columns={"UNDERDOG": "OFF_TEAM", "FAVORITE": "DEF_TEAM", "UNDERDOG_SCORE": "BETS_SCORE"})
test_df2['SPREAD'] = -test_df2['SPREAD']
test_df = pd.concat([test_df1, test_df2])

favorites_df = bets_df[['FAVORITE','FAVORITE_SCORE','SPREAD','WEEK']]
favorites_df.rename(columns={"FAVORITE": "Team", "FAVORITE_SCORE": "BETS_SCORE"}, inplace=True)

underdog_df = bets_df[['UNDERDOG','UNDERDOG_SCORE','SPREAD','WEEK']]
underdog_df['SPREAD'] = -underdog_df['SPREAD']
underdog_df.rename(columns={"UNDERDOG": "Team", "UNDERDOG_SCORE": "BETS_SCORE"}, inplace=True)

bets_df = pd.concat([favorites_df,underdog_df], ignore_index=True)
bets_df = pd.merge(bets_df, df_team_index, on='Team')
bets_df = bets_df[['Abb','WEEK','SPREAD','BETS_SCORE']]
bets_df.rename(columns={"Abb": "Team"}, inplace=True)
bets_df.head()

C:\Users\onine\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-1-02c53e2a8cf7>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  underdog_df['SPREAD'] = -underdog_df['SPREAD']


,Team,WEEK,SPREAD,BETS_SCORE
0,TB,1,-9.0,35.25
1,TB,2,-13.0,39.00
2,TB,3,-0.5,28.00
3,TB,4,-6.5,31.25
4,TB,5,-11.0,35.00


In [2]:
# Create variables for scraped data
OFF_TEAM = []
DEF_TEAM = []
FPTS = []
WEEK = []
pos = 'QB'
pos_dict = {'QB':1,'RB':2,'WR':3,'TE':4,'K':7,'DEF':8}
pos_key = pos_dict[pos]

for i in range(week_num):

    url = "https://fantasy.nfl.com/research/pointsagainst?position=" + str(pos_key) + "&statCategory=pointsAgainst&statSeason=2021&statType=weekPointsAgainst&statWeek=" + str(i+1)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    time.sleep(1)

    # Create an object of the first object that is class=data compact
    table = soup.find('tbody')

    # Find all the <tr> tag pairs, skip the first one, then for each.
    for row in table.find_all('tr'):
        
        # Create a variable of all the <td> tag pairs in each <tr> tag pair,
        #col = row.find('td').text

        DEF_TEAM.append(row.find_all('td')[0].text)
        OFF_TEAM.append(row.find_all('td')[1].text)
        FPTS.append(row.find_all('td')[2].text)
        WEEK.append(i+1)

    # Create a variable of the value of the columns
col = {'OFF_TEAM': OFF_TEAM,
       'DEF_TEAM': DEF_TEAM,
       'FPTS': FPTS,
       'WEEK': WEEK,
       }

    # Create a dataframe from the columns variable
df = pd.DataFrame(col)
df['POSITION'] = df['DEF_TEAM'].apply(lambda x: x.split('vs ')[1])
df['DEF_TEAM'] = df['DEF_TEAM'].apply(lambda x: x.split('vs ')[0]).str.replace(" Defense", "")
df['OFF_TEAM'] = df['OFF_TEAM'].apply(lambda x: x.replace("@", ""))
df['OFF_TEAM'] = df['OFF_TEAM'].apply(lambda x: 'WSH' if x=='WAS' else x)
df['OFF_TEAM'] = df['OFF_TEAM'].apply(lambda x: 'LAR' if x=='LA' else x)
df['FPTS'] = df['FPTS'].astype(float).astype(int)
df = df[["WEEK", "OFF_TEAM", "DEF_TEAM" , "FPTS"]]

df.to_csv("FF_Data", index=False)
df = pd.merge(df, df_team_index, left_on='DEF_TEAM', right_on='FullName')
df.drop(['DEF_TEAM'], axis=1, inplace=True)
df.rename(columns={"Abb": "DEF_TEAM"}, inplace=True)
OFF_df = df.groupby(by='OFF_TEAM').mean().reset_index()[['OFF_TEAM','FPTS']]
DEF_df = df.groupby(by='DEF_TEAM').mean().reset_index()[['DEF_TEAM','FPTS']]
df = pd.merge(df, OFF_df, on='OFF_TEAM', suffixes=[None,"_OFF_AVG"])
df = pd.merge(df, DEF_df, on='DEF_TEAM', suffixes=[None,"_DEF_AVG"])
df2 = pd.merge(bets_df, df, right_on=['OFF_TEAM','WEEK'], left_on=['Team','WEEK'], how='inner')
df2 = df2[['WEEK','OFF_TEAM','FPTS_OFF_AVG','DEF_TEAM','FPTS_DEF_AVG','SPREAD','BETS_SCORE','FPTS']]
df2.head(3)

,WEEK,OFF_TEAM,FPTS_OFF_AVG,DEF_TEAM,FPTS_DEF_AVG,SPREAD,BETS_SCORE,FPTS
0,1,TB,25.125,DAL,16.375,-9.0,35.25,27
1,2,TB,25.125,ATL,18.000,-13.0,39.00,29
2,3,TB,25.125,LAR,16.375,-0.5,28.00,28


In [51]:
from catboost import CatBoostRegressor

X = df2[['FPTS_OFF_AVG','FPTS_DEF_AVG','SPREAD','BETS_SCORE']]
#X = df2[['FPTS_OFF_AVG','FPTS_DEF_AVG','SPREAD']]
y = df2['FPTS']

from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.7, random_state=1234)

#categorical_features_indices = np.where(X.dtypes != float)[0]

#importing library and building model
from catboost import CatBoostRegressor
model=CatBoostRegressor(iterations=1000, depth=4, learning_rate=0.01, loss_function='RMSE', use_best_model=True )
model.fit(X_train, y_train, eval_set=(X_validation, y_validation), plot=True) 
#model.fit(X_train, y_train, cat_features=categorical_features_indices, eval_set=(X_validation, y_validation),plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 8.2569533	test: 7.3987522	best: 7.3987522 (0)	total: 492us	remaining: 492ms
1:	learn: 8.2384212	test: 7.3852755	best: 7.3852755 (1)	total: 884us	remaining: 442ms
2:	learn: 8.2230833	test: 7.3775178	best: 7.3775178 (2)	total: 1.25ms	remaining: 416ms
3:	learn: 8.2036928	test: 7.3649255	best: 7.3649255 (3)	total: 1.61ms	remaining: 402ms
4:	learn: 8.1859152	test: 7.3523605	best: 7.3523605 (4)	total: 1.9ms	remaining: 378ms
5:	learn: 8.1648708	test: 7.3434314	best: 7.3434314 (5)	total: 2.17ms	remaining: 360ms
6:	learn: 8.1497478	test: 7.3293621	best: 7.3293621 (6)	total: 2.51ms	remaining: 356ms
7:	learn: 8.1349579	test: 7.3181060	best: 7.3181060 (7)	total: 2.83ms	remaining: 350ms
8:	learn: 8.1126872	test: 7.3041202	best: 7.3041202 (8)	total: 4.14ms	remaining: 456ms
9:	learn: 8.0948138	test: 7.2937160	best: 7.2937160 (9)	total: 4.78ms	remaining: 474ms
10:	learn: 8.0770106	test: 7.2813021	best: 7.2813021 (10)	total: 5.2ms	remaining: 468ms
11:	learn: 8.0552452	test: 7.2683748	best: 7.

490:	learn: 5.7287099	test: 7.2690399	best: 6.8425857 (138)	total: 135ms	remaining: 140ms
491:	learn: 5.7257074	test: 7.2697885	best: 6.8425857 (138)	total: 136ms	remaining: 140ms
492:	learn: 5.7226396	test: 7.2698982	best: 6.8425857 (138)	total: 136ms	remaining: 140ms
493:	learn: 5.7202114	test: 7.2694223	best: 6.8425857 (138)	total: 136ms	remaining: 139ms
494:	learn: 5.7183351	test: 7.2718676	best: 6.8425857 (138)	total: 136ms	remaining: 139ms
495:	learn: 5.7169705	test: 7.2717028	best: 6.8425857 (138)	total: 137ms	remaining: 139ms
496:	learn: 5.7153756	test: 7.2731453	best: 6.8425857 (138)	total: 137ms	remaining: 138ms
497:	learn: 5.7116830	test: 7.2752740	best: 6.8425857 (138)	total: 137ms	remaining: 138ms
498:	learn: 5.7091488	test: 7.2755892	best: 6.8425857 (138)	total: 137ms	remaining: 138ms
499:	learn: 5.7080048	test: 7.2762836	best: 6.8425857 (138)	total: 138ms	remaining: 138ms
500:	learn: 5.7064649	test: 7.2772818	best: 6.8425857 (138)	total: 138ms	remaining: 137ms
501:	learn

In [52]:
test_df2 = pd.merge(test_df, df_team_index, left_on='OFF_TEAM', right_on='Team')
test_df2.drop(['OFF_TEAM'], axis=1, inplace=True)
test_df2.rename(columns={"Abb": "OFF_TEAM"}, inplace=True)
test_df2 = pd.merge(test_df2, df_team_index, left_on='DEF_TEAM', right_on='Team')
test_df2.drop(['DEF_TEAM'], axis=1, inplace=True)
test_df2.rename(columns={"Abb": "DEF_TEAM"}, inplace=True)

test_df2 = pd.merge(test_df2, OFF_df, on='OFF_TEAM', suffixes=[None,"_OFF_AVG"])
test_df2 = pd.merge(test_df2, DEF_df, on='DEF_TEAM', suffixes=[None,"_DEF_AVG"])
test_df2.rename(columns={"FPTS": "FPTS_OFF_AVG"}, inplace=True)

test_df2 = test_df2[['OFF_TEAM','FPTS_OFF_AVG','DEF_TEAM','FPTS_DEF_AVG','SPREAD','BETS_SCORE']]
test_df3 = test_df2[['FPTS_OFF_AVG','FPTS_DEF_AVG','SPREAD','BETS_SCORE']]
#test_df3 = test_df2[['FPTS_OFF_AVG','FPTS_DEF_AVG','SPREAD']]

test_df2['pred'] = model.predict(test_df3)
test_df2['POS'] = pos
test_df2.sort_values(by='pred', ascending=False).head(50)

,OFF_TEAM,FPTS_OFF_AVG,DEF_TEAM,FPTS_DEF_AVG,SPREAD,BETS_SCORE,pred,POS
12,LAR,22.375000,TEN,19.750,-7.5,34.25,22.430252,QB
7,BUF,27.428571,JAX,17.625,-14.5,39.00,22.011906,QB
1,CIN,19.625000,CLE,20.250,-2.5,25.75,21.282040,QB
24,GB,19.000000,KC,20.750,2.5,25.00,19.860953,QB
4,NO,21.857143,ATL,18.000,-6.0,27.25,19.648807,QB
11,ARI,21.750000,SF,18.500,-2.5,26.25,19.501635,QB
10,KC,20.125000,GB,17.000,-2.5,30.00,18.724784,QB
3,MIA,16.750000,HOU,20.125,-7.0,30.25,18.567524,QB
9,LAC,20.571429,PHI,16.000,-2.5,27.75,18.493760,QB
8,BAL,23.571429,MIN,14.250,-5.5,30.25,18.313713,QB


In [13]:
from sklearn.model_selection import cross_val_score
model=CatBoostRegressor(iterations=1000, depth=4, learning_rate=0.01, loss_function='RMSE')
accuracies = cross_val_score(estimator = model, X = X_train, y = y_train, cv = 10)
print("Accuracy:{:.2f}".format(accuracies.mean()*100))
print("Standard Deviation:{:.2f}".format(accuracies.std()))

0:	learn: 4.0962117	total: 312us	remaining: 313ms
1:	learn: 4.0882941	total: 742us	remaining: 370ms
2:	learn: 4.0832173	total: 1.04ms	remaining: 348ms
3:	learn: 4.0763662	total: 1.34ms	remaining: 334ms
4:	learn: 4.0686795	total: 1.58ms	remaining: 314ms
5:	learn: 4.0630158	total: 1.85ms	remaining: 307ms
6:	learn: 4.0567193	total: 2.15ms	remaining: 306ms
7:	learn: 4.0495357	total: 2.52ms	remaining: 313ms
8:	learn: 4.0425572	total: 2.85ms	remaining: 314ms
9:	learn: 4.0355073	total: 3.24ms	remaining: 321ms
10:	learn: 4.0277082	total: 3.56ms	remaining: 320ms
11:	learn: 4.0200879	total: 4.1ms	remaining: 337ms
12:	learn: 4.0153412	total: 4.49ms	remaining: 341ms
13:	learn: 4.0096161	total: 4.84ms	remaining: 341ms
14:	learn: 4.0037195	total: 5.13ms	remaining: 337ms
15:	learn: 3.9960395	total: 5.42ms	remaining: 333ms
16:	learn: 3.9882128	total: 5.71ms	remaining: 330ms
17:	learn: 3.9801610	total: 6.03ms	remaining: 329ms
18:	learn: 3.9748507	total: 6.31ms	remaining: 326ms
19:	learn: 3.9677460	tota

592:	learn: 2.7848334	total: 149ms	remaining: 102ms
593:	learn: 2.7837339	total: 149ms	remaining: 102ms
594:	learn: 2.7827733	total: 150ms	remaining: 102ms
595:	learn: 2.7817286	total: 150ms	remaining: 102ms
596:	learn: 2.7809499	total: 150ms	remaining: 101ms
597:	learn: 2.7803831	total: 150ms	remaining: 101ms
598:	learn: 2.7798898	total: 151ms	remaining: 101ms
599:	learn: 2.7789273	total: 151ms	remaining: 101ms
600:	learn: 2.7757037	total: 151ms	remaining: 100ms
601:	learn: 2.7749548	total: 151ms	remaining: 100ms
602:	learn: 2.7740674	total: 152ms	remaining: 99.9ms
603:	learn: 2.7734678	total: 152ms	remaining: 99.6ms
604:	learn: 2.7728014	total: 152ms	remaining: 99.4ms
605:	learn: 2.7723409	total: 152ms	remaining: 99.1ms
606:	learn: 2.7709299	total: 153ms	remaining: 98.8ms
607:	learn: 2.7696190	total: 153ms	remaining: 98.6ms
608:	learn: 2.7684479	total: 153ms	remaining: 98.3ms
609:	learn: 2.7678456	total: 153ms	remaining: 98.1ms
610:	learn: 2.7670131	total: 154ms	remaining: 97.8ms
611

111:	learn: 3.5762422	total: 27.6ms	remaining: 219ms
112:	learn: 3.5732638	total: 28ms	remaining: 219ms
113:	learn: 3.5697237	total: 28.2ms	remaining: 219ms
114:	learn: 3.5650609	total: 28.5ms	remaining: 219ms
115:	learn: 3.5631357	total: 28.7ms	remaining: 218ms
116:	learn: 3.5600018	total: 28.9ms	remaining: 218ms
117:	learn: 3.5564045	total: 29.2ms	remaining: 218ms
118:	learn: 3.5531617	total: 29.5ms	remaining: 218ms
119:	learn: 3.5492272	total: 29.7ms	remaining: 218ms
120:	learn: 3.5457373	total: 30ms	remaining: 218ms
121:	learn: 3.5438533	total: 30.2ms	remaining: 217ms
122:	learn: 3.5398195	total: 30.4ms	remaining: 217ms
123:	learn: 3.5361478	total: 30.6ms	remaining: 216ms
124:	learn: 3.5333671	total: 30.9ms	remaining: 216ms
125:	learn: 3.5302541	total: 31.2ms	remaining: 216ms
126:	learn: 3.5284751	total: 31.5ms	remaining: 216ms
127:	learn: 3.5259817	total: 31.8ms	remaining: 216ms
128:	learn: 3.5217706	total: 32ms	remaining: 216ms
129:	learn: 3.5185110	total: 32.2ms	remaining: 216ms

690:	learn: 2.6934911	total: 168ms	remaining: 75.3ms
691:	learn: 2.6928673	total: 169ms	remaining: 75.1ms
692:	learn: 2.6923488	total: 169ms	remaining: 74.8ms
693:	learn: 2.6918876	total: 169ms	remaining: 74.6ms
694:	learn: 2.6908692	total: 169ms	remaining: 74.3ms
695:	learn: 2.6904657	total: 170ms	remaining: 74.2ms
696:	learn: 2.6901626	total: 170ms	remaining: 74ms
697:	learn: 2.6881502	total: 170ms	remaining: 73.7ms
698:	learn: 2.6877072	total: 171ms	remaining: 73.5ms
699:	learn: 2.6873697	total: 171ms	remaining: 73.3ms
700:	learn: 2.6868969	total: 171ms	remaining: 73ms
701:	learn: 2.6855278	total: 171ms	remaining: 72.8ms
702:	learn: 2.6844600	total: 172ms	remaining: 72.5ms
703:	learn: 2.6828383	total: 172ms	remaining: 72.3ms
704:	learn: 2.6811202	total: 172ms	remaining: 72ms
705:	learn: 2.6793795	total: 172ms	remaining: 71.8ms
706:	learn: 2.6783250	total: 173ms	remaining: 71.5ms
707:	learn: 2.6766588	total: 173ms	remaining: 71.3ms
708:	learn: 2.6750461	total: 173ms	remaining: 71ms
7

222:	learn: 3.3142454	total: 54.1ms	remaining: 188ms
223:	learn: 3.3130190	total: 54.4ms	remaining: 189ms
224:	learn: 3.3099102	total: 54.7ms	remaining: 188ms
225:	learn: 3.3073869	total: 54.9ms	remaining: 188ms
226:	learn: 3.3061293	total: 55.2ms	remaining: 188ms
227:	learn: 3.3046715	total: 55.5ms	remaining: 188ms
228:	learn: 3.3036215	total: 55.7ms	remaining: 187ms
229:	learn: 3.3016114	total: 55.9ms	remaining: 187ms
230:	learn: 3.3004787	total: 56.2ms	remaining: 187ms
231:	learn: 3.2985314	total: 56.5ms	remaining: 187ms
232:	learn: 3.2969937	total: 56.8ms	remaining: 187ms
233:	learn: 3.2953926	total: 57ms	remaining: 187ms
234:	learn: 3.2938580	total: 57.2ms	remaining: 186ms
235:	learn: 3.2930376	total: 57.5ms	remaining: 186ms
236:	learn: 3.2919729	total: 57.7ms	remaining: 186ms
237:	learn: 3.2905599	total: 58.1ms	remaining: 186ms
238:	learn: 3.2891333	total: 58.4ms	remaining: 186ms
239:	learn: 3.2868399	total: 58.6ms	remaining: 186ms
240:	learn: 3.2856307	total: 58.9ms	remaining: 1

...

754:	learn: 2.6070296	total: 187ms	remaining: 60.6ms
755:	learn: 2.6059345	total: 187ms	remaining: 60.4ms
756:	learn: 2.6057526	total: 187ms	remaining: 60.2ms
757:	learn: 2.6053850	total: 188ms	remaining: 59.9ms
758:	learn: 2.6047452	total: 188ms	remaining: 59.7ms
759:	learn: 2.6029116	total: 188ms	remaining: 59.4ms
760:	learn: 2.6025330	total: 188ms	remaining: 59.2ms
761:	learn: 2.6017321	total: 189ms	remaining: 59ms
762:	learn: 2.6011711	total: 189ms	remaining: 58.7ms
763:	learn: 2.6005920	total: 189ms	remaining: 58.5ms
764:	learn: 2.6002343	total: 189ms	remaining: 58.2ms
765:	learn: 2.5978668	total: 190ms	remaining: 57.9ms
766:	learn: 2.5972371	total: 190ms	remaining: 57.7ms
767:	learn: 2.5942065	total: 190ms	remaining: 57.4ms
768:	learn: 2.5918893	total: 190ms	remaining: 57.2ms
769:	learn: 2.5917286	total: 191ms	remaining: 56.9ms
770:	learn: 2.5911328	total: 191ms	remaining: 56.7ms
771:	learn: 2.5905425	total: 191ms	remaining: 56.4ms
772:	learn: 2.5901315	total: 191ms	remaining: 56

273:	learn: 3.2425615	total: 65.9ms	remaining: 175ms
274:	learn: 3.2396221	total: 66.3ms	remaining: 175ms
275:	learn: 3.2384770	total: 66.5ms	remaining: 174ms
276:	learn: 3.2361551	total: 66.7ms	remaining: 174ms
277:	learn: 3.2343392	total: 67ms	remaining: 174ms
278:	learn: 3.2337554	total: 67.3ms	remaining: 174ms
279:	learn: 3.2317099	total: 67.6ms	remaining: 174ms
280:	learn: 3.2293221	total: 67.9ms	remaining: 174ms
281:	learn: 3.2279919	total: 68.2ms	remaining: 174ms
282:	learn: 3.2262492	total: 68.5ms	remaining: 173ms
283:	learn: 3.2247662	total: 68.7ms	remaining: 173ms
284:	learn: 3.2218982	total: 69ms	remaining: 173ms
285:	learn: 3.2215311	total: 69.1ms	remaining: 173ms
286:	learn: 3.2198137	total: 69.4ms	remaining: 172ms
287:	learn: 3.2180545	total: 69.6ms	remaining: 172ms
288:	learn: 3.2167490	total: 69.9ms	remaining: 172ms
289:	learn: 3.2146339	total: 70.2ms	remaining: 172ms
290:	learn: 3.2116954	total: 70.4ms	remaining: 172ms
291:	learn: 3.2112106	total: 70.7ms	remaining: 171

819:	learn: 2.5705247	total: 203ms	remaining: 44.6ms
820:	learn: 2.5701429	total: 203ms	remaining: 44.4ms
821:	learn: 2.5684054	total: 204ms	remaining: 44.1ms
822:	learn: 2.5672050	total: 204ms	remaining: 43.9ms
823:	learn: 2.5658532	total: 204ms	remaining: 43.6ms
824:	learn: 2.5656069	total: 205ms	remaining: 43.4ms
825:	learn: 2.5636117	total: 205ms	remaining: 43.2ms
826:	learn: 2.5632268	total: 205ms	remaining: 42.9ms
827:	learn: 2.5628522	total: 205ms	remaining: 42.7ms
828:	learn: 2.5622588	total: 206ms	remaining: 42.4ms
829:	learn: 2.5610139	total: 206ms	remaining: 42.2ms
830:	learn: 2.5607206	total: 206ms	remaining: 41.9ms
831:	learn: 2.5602660	total: 206ms	remaining: 41.7ms
832:	learn: 2.5593719	total: 207ms	remaining: 41.4ms
833:	learn: 2.5591346	total: 207ms	remaining: 41.2ms
834:	learn: 2.5588321	total: 207ms	remaining: 40.9ms
835:	learn: 2.5584360	total: 207ms	remaining: 40.7ms
836:	learn: 2.5582190	total: 208ms	remaining: 40.4ms
837:	learn: 2.5567843	total: 208ms	remaining: 

317:	learn: 3.1617258	total: 82.7ms	remaining: 177ms
318:	learn: 3.1603531	total: 83ms	remaining: 177ms
319:	learn: 3.1585710	total: 83.3ms	remaining: 177ms
320:	learn: 3.1572434	total: 83.6ms	remaining: 177ms
321:	learn: 3.1566224	total: 83.9ms	remaining: 177ms
322:	learn: 3.1553304	total: 84.1ms	remaining: 176ms
323:	learn: 3.1539418	total: 84.4ms	remaining: 176ms
324:	learn: 3.1511753	total: 84.7ms	remaining: 176ms
325:	learn: 3.1495093	total: 84.9ms	remaining: 176ms
326:	learn: 3.1474656	total: 85.2ms	remaining: 175ms
327:	learn: 3.1453863	total: 85.4ms	remaining: 175ms
328:	learn: 3.1418586	total: 85.7ms	remaining: 175ms
329:	learn: 3.1402890	total: 86.1ms	remaining: 175ms
330:	learn: 3.1394652	total: 86.3ms	remaining: 174ms
331:	learn: 3.1375049	total: 86.6ms	remaining: 174ms
332:	learn: 3.1363288	total: 86.8ms	remaining: 174ms
333:	learn: 3.1346218	total: 87.1ms	remaining: 174ms
334:	learn: 3.1330996	total: 87.3ms	remaining: 173ms
335:	learn: 3.1316017	total: 87.5ms	remaining: 1

886:	learn: 2.5303797	total: 227ms	remaining: 29ms
887:	learn: 2.5301519	total: 228ms	remaining: 28.7ms
888:	learn: 2.5298959	total: 228ms	remaining: 28.5ms
889:	learn: 2.5289806	total: 228ms	remaining: 28.2ms
890:	learn: 2.5287025	total: 229ms	remaining: 28ms
891:	learn: 2.5274108	total: 229ms	remaining: 27.7ms
892:	learn: 2.5271270	total: 229ms	remaining: 27.4ms
893:	learn: 2.5263650	total: 229ms	remaining: 27.2ms
894:	learn: 2.5250513	total: 230ms	remaining: 26.9ms
895:	learn: 2.5242331	total: 230ms	remaining: 26.7ms
896:	learn: 2.5238741	total: 230ms	remaining: 26.4ms
897:	learn: 2.5234144	total: 230ms	remaining: 26.2ms
898:	learn: 2.5221064	total: 231ms	remaining: 25.9ms
899:	learn: 2.5209230	total: 231ms	remaining: 25.7ms
900:	learn: 2.5186021	total: 231ms	remaining: 25.4ms
901:	learn: 2.5177100	total: 231ms	remaining: 25.1ms
902:	learn: 2.5168100	total: 232ms	remaining: 24.9ms
903:	learn: 2.5160236	total: 232ms	remaining: 24.6ms
904:	learn: 2.5150303	total: 232ms	remaining: 24.4